In [1]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from lags import create_df_with_lags

In [2]:
df = pd.read_csv("../processed_tables/merged_standardized.csv")

## Creación de lags para el precio de la energía

In [3]:
df = create_df_with_lags(df, 3)
lag_columns = df.filter(like="lag_")
price = df[["Date", "energy_price"]]
df = pd.concat([price, lag_columns], axis=1)
df.head()

,Date,energy_price,energy_price_lag_1,energy_price_lag_2,energy_price_lag_3
3,2021-10-04,286.526116,282.065065,256.940174,216.929726
4,2021-10-05,278.157325,286.526116,282.065065,256.940174
5,2021-10-06,261.474788,278.157325,286.526116,282.065065
6,2021-10-07,235.857848,261.474788,278.157325,286.526116
7,2021-10-08,229.313699,235.857848,261.474788,278.157325


## Partición de la data

Debido a que la naturaleza de la data es de carácter temporal, se realizará una partición de la data en 3 partes: entrenamiento y prueba. La data de test será los últimos 3 meses de la data, es decir desde julio 2023 hasta septiembre 2023.


In [4]:
fecha_corte = "2023-07-01"
X_train = df[df["Date"] < fecha_corte].drop(["Date", "energy_price"], axis=1)
X_test = df[df["Date"] >= fecha_corte].drop(["Date", "energy_price"], axis=1)
y_train = df[df["Date"] < fecha_corte]["energy_price"]
y_test = df[df["Date"] >= fecha_corte]["energy_price"]

## KNN

Esta vez utilizaremos `sklearn` para hacer KNN con regresión, variando parámetros como el número de vecinos, la métrica de distancia y la técnica de pesado.

In [5]:
param_dist = {
    'n_neighbors': range(1, 30),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Creating the RandomizedSearchCV object
knn_regressor = KNeighborsRegressor()
random_search = RandomizedSearchCV(knn_regressor, param_distributions=param_dist, 
                                   n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Fitting the model
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_iter=100,
                   param_distributions={'n_neighbors': range(1, 30),
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='neg_mean_squared_error')

In [6]:
print(f"Best parameters: {random_search.best_params_}")

Best parameters: {'weights': 'distance', 'p': 2, 'n_neighbors': 8}


In [7]:
best_knn_regressor = random_search.best_estimator_
y_pred = best_knn_regressor.predict(X_test)

In [8]:
mse_test = mean_squared_error(y_test, y_pred)

print("Mean Squared Error on Test Data:", mse_test)

Mean Squared Error on Test Data: 25836.433945829678
